![img](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/CRISP-DM_Process_Diagram.png/598px-CRISP-DM_Process_Diagram.png)

# Objective?

Get all the descriptions of the Products and their department

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random

## Links

We are gonna gather all the links from the sections that we want and when we find 'Agencia de viajes' We brake the loop because we dont want the links down it

In [2]:
url = 'https://www.elcorteingles.es/'

req = requests.get(url)

soupHome = BeautifulSoup(req.text)

i = 0
links = []

for a in soupHome.find_all('a', href=True):
    if ('viajeselcorteingles' in a['href']):
        break
    if all([term not in a['href'] for term in ['/supermercado/', '/entradas/', '/club-del-gourmet/']]):
        links.append(a['href'].split('?')[0])

#Remove duplicates
links = list(dict.fromkeys(links))

In [6]:
len(links)

1301

In [10]:
links[900]

'/juguetes/munecos-articulados/'

## Correcciones manuales

Estos links corresponden a listas de prodcutos que nos producen duplicados, si las filtramos de inicio optimizamos el proceso al tener que hacer menos peticiones.

In [5]:
links_to_remove = ['/75/','/moda/', '/moda/mujer/', '/moda/zapatos/', '/cine/', '/musica/', '/mascotas/', '/bricor/',
                  '/bricor/iluminacion/', '/bricor/herramientas/', '/bricor/estanterias-y-ordenacion/', '/juguetes/'
                  '/bricor/armarios/', '/electronica/', '/electrodomesticos/', '/deportes/', '/hogar/', '/libros/',
                  '/bricor/bano/','/bricor/cocinas/',' /bricor/armarios/', 'https://www.elcorteingles.es/perfumeria/']
for x in links_to_remove:
    try:
        links.remove(x)
    except Exception as e:
        print(f'Exception: {e}')
        

Exception: list.remove(x): x not in list
Exception: list.remove(x): x not in list
Exception: list.remove(x): x not in list


## Productos

In [11]:
products = []
url_errors = []
suffix = ''
    
for url in links[900:]:
    url = 'https://www.elcorteingles.es'+url #Juguetes juegos de mesa y habilidad
    print (url)
    for page in range (15,30):
        time.sleep(random.randint(0,1)+random.random()) #Para evitar suponerle un daño a la empresa
        try:
            if page > 1:
                suffix = '/'+str(page)+'/'
                
            req = requests.get(url+suffix, timeout=10)

            if req.ok: # status_code == 200
                s = BeautifulSoup(req.text)
                prods = s.findAll('span')

                for n, span_ in enumerate(prods):
                    if 'data-json' in span_.attrs and ('data-scope' in span_.attrs and span_.attrs['data-scope'] == 'product'):
                        obj = json.loads(span_['data-json'])
                        obj['image'] = 'http:{}'.format(str(span_.parent.find('img')['src']))
                        products.append(obj)

            else: # imprime url y status code donde ha dado error
                print(f'Status code error on url {url}. Status code: {req.status_code}')
                url_errors.append(url)
                break # Si no hay mas pages no sigue buscando mas y pasa al siguiente link
                
        except Exception as e:
            print(f'Exception: {e}')
            if url not in url_errors:
                url_errors.append(url)
                
    time.sleep(random.randint(5,10) + random.random()) #Para evitar suponerle un daño a la empresa
                


https://www.elcorteingles.es/juguetes/munecos-articulados/
https://www.elcorteingles.es/juguetes/munecas-y-peluches/
https://www.elcorteingles.es/juguetes/preescolar/
https://www.elcorteingles.es/juguetes/licencias-de-personajes/
Status code error on url https://www.elcorteingles.es/juguetes/licencias-de-personajes/. Status code: 400
https://www.elcorteingles.es/star-wars/
https://www.elcorteingles.es/tienda-disney/frozen/
Status code error on url https://www.elcorteingles.es/tienda-disney/frozen/. Status code: 400
https://www.elcorteingles.es/barbie/juguetes/
https://www.elcorteingles.es/juguetes/licencias-de-personajes/power-rangers/
Status code error on url https://www.elcorteingles.es/juguetes/licencias-de-personajes/power-rangers/. Status code: 400
https://www.elcorteingles.es/nuestras-marcas-juguetes/
https://www.elcorteingles.es/miyo/juguetes/
https://www.elcorteingles.es/eduland/juguetes/
https://www.elcorteingles.es/playland/juguetes/
https://www.elcorteingles.es/roll-run/jugu

https://www.elcorteingles.es/joie/bebes/
https://www.elcorteingles.es/maclaren/bebes/
https://www.elcorteingles.es/micuna/bebes/
https://www.elcorteingles.es/mountain-buggy/bebes/
https://www.elcorteingles.es/pasito-a-pasito/bebes/
https://www.elcorteingles.es/playxtrem/bebes/
https://www.elcorteingles.es/stokke/bebes/
https://www.elcorteingles.es/tuc-tuc/bebes/
https://www.elcorteingles.es/perfumeria/
https://www.elcorteingles.es/perfumeria/rostro-mujer/antiedad/
Status code error on url https://www.elcorteingles.es/perfumeria/rostro-mujer/antiedad/. Status code: 400
https://www.elcorteingles.es/perfumeria/cuerpo-mujer/
Status code error on url https://www.elcorteingles.es/perfumeria/cuerpo-mujer/. Status code: 400
https://www.elcorteingles.es/perfumeria/rostro-mujer/tratamientos-ojos/
Status code error on url https://www.elcorteingles.es/perfumeria/rostro-mujer/tratamientos-ojos/. Status code: 400
https://www.elcorteingles.es/perfumeria/fragancias-mujer/
https://www.elcorteingles.es/

https://www.elcorteingles.es/moda/joyeria-y-relojes/smartwatch/
Status code error on url https://www.elcorteingles.es/moda/joyeria-y-relojes/smartwatch/. Status code: 400
https://www.elcorteingles.es/moda/joyeria-y-relojes/relojes-infantiles/
Status code error on url https://www.elcorteingles.es/moda/joyeria-y-relojes/relojes-infantiles/. Status code: 400
https://www.elcorteingles.es/moda/joyeria-y-relojes/despertadores/
Status code error on url https://www.elcorteingles.es/moda/joyeria-y-relojes/despertadores/. Status code: 400
https://www.elcorteingles.es/moda/joyeria-y-relojes/accesorios-de-reloj/
Status code error on url https://www.elcorteingles.es/moda/joyeria-y-relojes/accesorios-de-reloj/. Status code: 400
https://www.elcorteingles.es/moda/accesorios/bisuteria/anillos/
Status code error on url https://www.elcorteingles.es/moda/accesorios/bisuteria/anillos/. Status code: 400
https://www.elcorteingles.es/moda/accesorios/bisuteria/colgantes/
https://www.elcorteingles.es/moda/acces

https://www.elcorteingles.es/libros/comics/superheroes/
https://www.elcorteingles.es/radar/comics/
Status code error on url https://www.elcorteingles.es/radar/comics/. Status code: 404
https://www.elcorteingles.es/papeleria/
https://www.elcorteingles.es/papeleria/agendas-y-calendarios/
https://www.elcorteingles.es/papeleria/archivo-y-clasificacion/
Status code error on url https://www.elcorteingles.es/papeleria/archivo-y-clasificacion/. Status code: 400
https://www.elcorteingles.es/papeleria/calculadoras-y-maquinas-de-oficina/
Status code error on url https://www.elcorteingles.es/papeleria/calculadoras-y-maquinas-de-oficina/. Status code: 400
https://www.elcorteingles.es/papeleria/cuadernos-libretas-y-recambios/
https://www.elcorteingles.es/papeleria/escritura-de-lujo/
https://www.elcorteingles.es/papeleria/escritura-y-correccion/
https://www.elcorteingles.es/papeleria/material-de-escritorio/
Status code error on url https://www.elcorteingles.es/papeleria/material-de-escritorio/. Statu

Exception: HTTPSConnectionPool(host='www.elcorteingles.es%20', port=443): Max retries exceeded with url: /mascotas/comida-para-perros//27/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000023792CF7940>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Exception: HTTPSConnectionPool(host='www.elcorteingles.es%20', port=443): Max retries exceeded with url: /mascotas/comida-para-perros//28/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000023792CF71D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
Exception: HTTPSConnectionPool(host='www.elcorteingles.es%20', port=443): Max retries exceeded with url: /mascotas/comida-para-perros//29/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000023792CF7DA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))
https://www.elcorteingles.es/mascotas/ac

Status code error on url https://www.elcorteingles.es/eukanuba-shop/mascotas/. Status code: 400
https://www.elcorteingles.es/pedigree/mascotas/
https://www.elcorteingles.es/nutro/mascotas/
https://www.elcorteingles.es/retorn/mascotas/
https://www.elcorteingles.es/whiskas/mascotas/
https://www.elcorteingles.es/gourmet-gold/mascotas/
https://www.elcorteingles.es/kong/mascotas/
https://www.elcorteingles.es/bricor/muebles-de-jardin/
Status code error on url https://www.elcorteingles.es/bricor/muebles-de-jardin/. Status code: 400
https://www.elcorteingles.es/bricor/muebles-de-jardin/barbacoas/
Status code error on url https://www.elcorteingles.es/bricor/muebles-de-jardin/barbacoas/. Status code: 400
https://www.elcorteingles.es/bricor/muebles-de-jardin-minisite/set-relax/
https://www.elcorteingles.es/bricor/muebles-de-jardin/mesas-comedor/mesas-comedor-extensibles/
Status code error on url https://www.elcorteingles.es/bricor/muebles-de-jardin/mesas-comedor/mesas-comedor-extensibles/. Status

https://www.elcorteingles.es/bricor/armarios/armarios-multiusos/
Status code error on url https://www.elcorteingles.es/bricor/armarios/armarios-multiusos/. Status code: 400
https://www.elcorteingles.es/bricor/armarios/mobiliario-dormitorio/
Status code error on url https://www.elcorteingles.es/bricor/armarios/mobiliario-dormitorio/. Status code: 400
https://www.elcorteingles.es/bricor/armarios/cajoneras-y-comodas/
Status code error on url https://www.elcorteingles.es/bricor/armarios/cajoneras-y-comodas/. Status code: 400
https://www.elcorteingles.es/bricor/bano/accesorios-de-bano/
Status code error on url https://www.elcorteingles.es/bricor/bano/accesorios-de-bano/. Status code: 400
https://www.elcorteingles.es/bricor/bano/griferia-monomando/
Status code error on url https://www.elcorteingles.es/bricor/bano/griferia-monomando/. Status code: 400
https://www.elcorteingles.es/bricor/bano/griferia-termostatica/search/
Status code error on url https://www.elcorteingles.es/bricor/bano/grifer

In [12]:
len(products)

66950

In [13]:
products[60000]

{'id': '001005120009195',
 'brand': 'HIGHNOTE',
 'store_id': '7',
 'position': 7,
 'badges': [],
 'media': {'count': 1},
 'name': 'Swings the Penthouse (CD)',
 'variant': '001005120009195   ',
 'category': ['Música', 'Jazz-Blues', 'Jazz'],
 'alternative_id': 'A14911814',
 'eci_provider': '00000000',
 'gtin': '0632375727321',
 'price': {'final': 17.8, 'currency': 'EUR'},
 'status': 'add',
 'quantity': 1,
 'image': 'http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA02/CONTENIDOS/201503/18/00105120009195____1__640x640.jpg'}

Lo guardamos en un Dataframe para poder usarlo en batch

In [14]:
dfProducts = pd.DataFrame.from_records(products)
dfProducts.head(3)

,alternative_id,availability_date,badges,brand,category,discount,eci_provider,gtin,id,image,media,module_name,name,position,price,quantity,status,store_id,variant,vendor
0,A17665205,NaN,[express_delivery],Schleich,"[Juguetes, Muñecos articulados, Figuras y masc...",NaN,00000000,4005086137981,001002712248919,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},NaN,Ternero frisón de manchas negras Schleich,0.0,"{'final': 4.99, 'currency': 'EUR'}",1,add,38,001002712248919,NaN
1,A17665214,NaN,[express_delivery],Schleich,"[Juguetes, Muñecos articulados, Figuras y masc...",NaN,00000000,4005086145757,001002712248935,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},NaN,Dunkleosteus Schleich,1.0,"{'final': 14.99, 'currency': 'EUR'}",1,add,38,001002712248935,NaN
2,A17665229,NaN,[],Schleich,"[Juguetes, Muñecos articulados, Figuras y masc...",NaN,00000000,4055744007316,001002712248968,http://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03...,{'count': 1},NaN,Pangolín gigante Schleich,2.0,"{'final': 4.99, 'currency': 'EUR'}",1,add,38,001002712248968,NaN


In [15]:
dfProducts.shape

(66950, 20)

### Quitar duplicados

In [16]:
dfProducts = dfProducts.drop_duplicates(subset='id')
dfProducts.shape

(40845, 20)

In [19]:
dfProducts.to_csv('Output/ProductsV4.csv', index=False)

## Save images

In [24]:
for n, product in enumerate (products):
    pic_url = product['image']
    print (n)
    
    name = ''
    for i in product['category']:
        name += i+'_'
    name = name + product['id'] + '.jpg'
    
    with open('Images/'+name, 'wb') as img_file:
        response = requests.get(pic_url, stream=True)

        if response.ok:

            for batch in response.iter_content(1024):
                if not batch:
                    break

                img_file.write(batch)
        else:
            print(f'Status code error with {response.status_code} code.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

Top 20 Marcas

In [78]:
dfProducts['brand'].value_counts()[:20]

El Corte Inglés         5062
Tommy Hilfiger          1421
Freestyle               1087
Polo Ralph Lauren       1070
Emidio Tucci            1043
Nike                    1026
Fox Home                 918
Gloria Ortiz             830
Dustin                   824
MICHAEL Michael Kors     804
Easy Wear                777
Dulces                   757
GAP                      725
Tous                     713
Sony                     697
Brotes                   692
Tutto Piccolo            674
Tuc tuc                  674
Énfasis                  664
Philips                  645
Name: brand, dtype: int64